In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import scipy.stats as sc
from datetime import datetime
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('./Road Accident/Road-Accident.csv', na_values=-1, parse_dates=['time'], infer_datetime_format=True)
# print(data.head())

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# config datetime to ordered integers
st = pd.Timestamp(year=1900, month=1, day=1)
def convert_date(x):
    return (pd.to_datetime(x,dayfirst=True) - st).days + 2
#print((pd.to_datetime('31/12/2014',dayfirst=True) - st).days + 2)
for i, row in data.iterrows():
    if len(row['date']) == 10:
        c = convert_date(row['date'])
        data.set_value(i,'date', c)
        


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


In [13]:
data['date'] = data['date'].astype(np.int64)
data.index = data.date
data = data.sort_index()

In [14]:
nan_values = [[c, data[c].isnull().sum()/data.shape[0]] for c in data.columns.values]
for n in nan_values:
    if n[1] > 0.4:
        print(n)
        
# we are removing columns with str types   
data = data.select_dtypes(exclude=['O'])
        
print(data.shape)

# we are dropping columns with more than 40% missing values
for i in nan_values:
    if i[1] > 0.4:
        data.drop(i[0],axis=1, inplace=True)
print(data.shape)

# dropping rows with nan values
data = data.dropna()

print(data.shape)


(66009, 63)
(66009, 63)
(66009, 63)


In [43]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot
 
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values
 
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y
 
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy
 
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 0, 25, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape data into time steps of sub-sequences
	n_steps, n_length = 4, 32
	trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
	# define model
	model = Sequential()
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
	model.add(TimeDistributed(Dropout(0.5)))
	model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
	model.add(TimeDistributed(Flatten()))
	model.add(LSTM(100))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy
 
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run an experiment
def run_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
 
# run the experiment
run_experiment()

Using TensorFlow backend.


FileNotFoundError: File b'HARDataset/train/Inertial Signals/total_acc_x_train.txt' does not exist

In [15]:
# using timeseries classification using rnn

import tensorflow as tf
from scipy import stats
from tensorflow.python.ops import rnn, rnn_cell
from sklearn.metrics import roc_auc_score

In [16]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)
        
def extract_segments(data, window_size = 30):
    segments = np.empty((0,(window_size + 1)))
    labels = np.empty((0))
    for (start,end) in windows(data,window_size):
        if(len(data.ix[start:end]) == (window_size + 1)):
            signal = data.ix[start:end]["<FEATURE COLUMN NAME>"]
            segments = np.vstack([segments, signal])
            labels = np.append(labels,stats.mode(data.ix[start:end]["<CLASS COLUMN NAME>"])[0][0])
    return segments, labels

In [17]:
win_size = 10
'''
MIMIC-III dataset can possibly be use to train and test the model. 
But beware this is not the data set used by the authors of the paper. 
For dataset description and format please see Section 3: Data Description in the paper.
'''

segments,labels = extract_segments(data, win_size)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
reshaped_segments = segments.reshape([len(segments),(win_size + 1),1])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


In [18]:
train_test_split = np.random.rand(len(reshaped_segments)) < 0.80
train_x = reshaped_segments[train_test_split]
train_y = labels[train_test_split]
test_x = reshaped_segments[~train_test_split]
test_y = labels[~train_test_split]


In [29]:
# Hyperparameter configuration

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 100
batch_size = 10
total_batches = (train_x.shape[0]//batch_size)

n_input = 1
n_steps = 10
n_hidden = 64
n_classes = 3

alpha = 0.5

In [30]:
# Input/output placeholders for TensorFlow graph

x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])
y_steps = tf.placeholder("float", [None, n_classes])

In [41]:
# Helper methods
# Addition of Dropout and/or other modification to model architecture can be made in LSTM function.

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def LSTM(x, weight, bias):
    cell = rnn_cell.LSTMCell(n_hidden,state_is_tuple = True)
    multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell([cell] * 2)
    output, state = tf.nn.dynamic_rnn(multi_layer_cell, x, dtype = tf.float32)
    output_flattened = tf.reshape(output, [-1, n_hidden])
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
    output_all = tf.nn.sigmoid(output_logits)
    output_reshaped = tf.reshape(output_all,[-1,n_steps,n_classes])
    output_last = tf.gather(tf.transpose(output_reshaped,[1,0,2]), n_steps - 1)  
    #output = tf.transpose(output, [1, 0, 2])
    #last = tf.gather(output, int(output.get_shape()[0]) - 1)
    #output_last = tf.nn.sigmoid(tf.matmul(last, weight) + bias)
    return output_last, output_all


def RNN(x, weights, biases):
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 1-layer LSTM with n_hidden units.
    cell = rnn_cell.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn_cell.static_rnn(cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [42]:
weight = weight_variable([n_hidden,n_classes])
bias = bias_variable([n_classes])
print(x.shape)
print(weight.shape)
print(bias.shape)
# y_last, y_all = LSTM(x,weight,bias)
y_last = RNN(x,weight,bias)


(?, 10, 1)
(64, 3)
(3,)


AttributeError: module 'tensorflow.python.ops.rnn_cell' has no attribute 'static_rnn'